# Cylinder Models

In this section, we describe models of intra-axonal diffusion.
In all cases, the intra-axonal diffusion is represented using axially symmetric cylinder models with $\boldsymbol{\mu}\in\mathbb{S}^2$ the orientation parallel to the cylinder axis.
The three-dimensional diffusion signal in these models is given as the separable product of (free) parallel and restricted perpendicular diffusion~\citep{assaf2004new}.
This means that the three-dimensional signal is given by

\begin{equation}\label{eq:cyl3d}
 E_{\textrm{intra}}(\textbf{q},\Delta,\delta,\lambda_\parallel,R) = E_\parallel(q_\parallel,\Delta,\delta,\lambda_\parallel)\times E_\perp(q_\perp,\Delta,\delta,R)
\end{equation}

with parallel q-value $q_\parallel=\textbf{q}^T\boldsymbol{\mu}$, perpendicular q-value $q_\perp=(\textbf{q}^T\textbf{q}-(\textbf{q}^T\boldsymbol{\mu})^2))^{1/2}$, parallel diffusivity $\lambda_\parallel>0$ and cylinder radius $R>0$[mm]. The parallel signal is usually given by Gaussian diffusion as

\begin{equation}
E_\parallel(q_\parallel,\Delta,\delta,\lambda_\parallel)=\exp(-4\pi^2q_\parallel^2\lambda_\parallel(\Delta-\delta/3)).
\end{equation}

The perpendicular signal $E_\perp$ is described using various cylinder models.
In the rest of this section, we start with describing the simplest, having the strongest tissue assumptions (I1), and more towards more general models (I4).
We end with a description how axon diameter distributions are modeled (DD1).

## Stick: C1
The simplest model for intra-axonal diffusion is the ``Stick'' -- a cylinder with zero radius~\citep{behrens2003characterization}.
The Stick model assumes that, because axon diameters are very small, the perpendicular diffusion attenuation inside these axons is negligible compared to the overall signal attenuation.
The perpendicular diffusion coefficient is therefore be approximated by zero, so the perpendicular signal attenuation is always equal to one as $E_\perp=1$.
Inserting this definition in Eq. (\ref{eq:cyl3d}) leads to the simple signal representation

\begin{equation}
E_{\textrm{Stick}}(b,\textbf{n},\boldsymbol{\mu},\lambda_\parallel)=\exp(-b\lambda_\parallel(\textbf{n}^T\boldsymbol{\mu})^2),
\end{equation}

which is the same as a DTI Tensor with $\lambda_\parallel=\lambda_1$ and $\lambda_\perp=\lambda_2=\lambda_3=0$.
Despite its simplicity, it turns out approximating axons as Sticks is quite reasonable at clinical gradient strengths~\citep{burcaw2015mesoscopic}.
In fact, the Stick is used in the most state-of-the-art microstructure models modeling axonal dispersion~\citep{tariq2016bingham,kaden2016multi}, which we'll get back to in Section \ref{sec:considerations}.

In [2]:
from microstruktur.signal_models import cylinder_models
stick = cylinder_models.C1Stick()

## Soderman Cylinder: C2
In reality, axons have a non-zero radius.
To account for this, different cylinder models for perpendicular diffusion have been proposed for different combinations of PGSE acquisition parameters.
The simplest is the ``Soderman'' model~\citep{soderman1995restricted}, which has the hardest assumptions on the PGSE protocol.
First, it assumes that pulse length $\delta$ is so short that no diffusion occurs during the application of the gradient pulse ($\delta\rightarrow0$).
%Recalling work \ref{sec:p1c1:dmri}, this allows us to interpret the diffusion propagator as the Fourier transform of the diffusion signal attenuation $E(\textbf{q},\Delta)$~\citep{tanner1968restricted}.
Second, it assumes that pulse separation $\Delta$ is long enough for diffusion with intra-cylindrical diffusion coefficient $D$ to be restricted inside a cylinder of radius $R$ ($\Delta\gg R^2/D$).
Within these assumptions, the perpendicular, intra-cylindrical signal attenuation is given as

\begin{equation}
 E_\perp(q,R|\delta\rightarrow0,\Delta\gg R^2/D)=\left(\frac{J_1(2\pi q R)}{\pi q R}\right)^2,
\end{equation}

where we use the ``$|$'' to separate function parameters from model assumptions, and $J_1$ is a Bessel function of the first kind. Taking $\lim_{R\rightarrow0}$ of Eq. (\ref{eq:soderman}) simplifies the three-dimensional Soderman model to the Stick model as in Eq. (\ref{eq:stick}).

In [3]:
soderman = cylinder_models.C2CylinderSodermanApproximation()

## Callaghan Cylinder: C3
The ``Callaghan'' model relaxes Soderman's $\Delta\gg R^2/D$ assumption to allow for unrestricted diffusion at shorter pulse separation $\Delta$~\citep{callaghan1995pulsed}. In this case, the perpendicular signal attenuation is given as

\begin{align}
 E_\perp(q,\Delta,R|\delta\rightarrow0)&=\sum^\infty_k4\exp(-\beta^2_{0k}D\Delta/R^2)\times \frac{\left((2\pi qR)J_0^{'}(2\pi qR)\right)^2}{\left((2\pi qR)^2-\beta_{0k}^2\right)^2}\nonumber\\
 &+\sum^\infty_{nk}8\exp(-\beta^2_{nk}D\Delta/R^2)\times \frac{\beta^2_{nk}}{\left(\beta_{nk}^2-n^2\right)}\times\frac{\left((2\pi qR)J_n^{'}(2\pi qR)\right)^2}{\left((2\pi qR)^2-\beta_{nk}^2\right)^2}
\end{align}

where $J_n^{'}$ are the derivatives of the $n^{th}$-order Bessel function and $\beta_{nk}$ are the arguments that result in zero-crossings. Taking $\lim_{\Delta\rightarrow\infty}$ of Eq. (\ref{eq:callaghan}) simplifies the Callaghan model to the Soderman model as in Eq. (\ref{eq:soderman}). The Callaghan model has been used to estimate the axon diameter distribution in the multi-compartment AxCaliber approach~\citep{assaf2008axcaliber}. However, the authors also mention that the perpendicular diffusion is likely already restricted for realistic axon diameters ($<2\mu$m)~\citep{aboitiz1992fiber} for the shortest possible $\Delta$ in PGSE protocols (${\sim}10$ms). This limits the added value of the Callaghan model over the Soderman model in axon diameter estimation.

In [4]:
callaghan = cylinder_models.C3CylinderCallaghanApproximation()

## Van Gelderen Cylinder: C4
The last cylinder model generalization we discuss is the ``Van Gelderen'' model~\citep{vangelderen1994evaluation}, which relaxes the last $\delta\rightarrow0$ assumption to allow for finite pulse length $\delta$. This model is based on the ``Neuman'' model~\citep{neuman1974spin}, which assumes Gaussian diffusion during the gradient pulse. In this case, the signal attenuation is given as

\begin{equation}
 E_\perp(q,\Delta,\delta,R)=-8\pi^2q^2\sum^\infty_{m=1}\dfrac{\left[\splitdfrac{2Da_m^2\delta-2 + 2e^{-Da_m^2\delta} + 2e^{-Da_m^2\Delta}}{-e^{-Da_m^2(\Delta-\delta)}-e^{-Da_m^2(\Delta-\delta)}}\right]}{\delta^2D^2a_m^6(R^2a_m^2-1)}
\end{equation}

where $a_m$ are roots of the equation $J_1^{'}(a_mR)=0$, with $J_1^{'}$ again the derivative of the Bessel function of the first kind, and $D$ is the intra-axonal diffusivity.
According to~\citet{neuman1974spin}, taking the double $\lim_{(\delta,\Delta)\rightarrow(0,\infty)}$ of Eq. (\ref{eq:vangelderen}) should simplify the Van Gelderen model to the Soderman Model in Eq. (\ref{eq:soderman}), although he does not show this explicitly.
For its generality, the Van Gelderen model has been used in most recent studies regarding \emph{in-vivo} axon diameter estimation~\citep{huang2015impact,ferizi2015white,de2016including}.

In [5]:
vangelderen = cylinder_models.C4CylinderGaussianPhaseApproximation()